In [ ]:
import collections
import cProfile
import hashlib
import itertools
import os
import pstats
import string
import re

from datetime import datetime
from io import StringIO
from tqdm.notebook import tqdm as tqdm

import torch
import torchaudio
from torch import nn, topk
from torch.optim import Adadelta
from torch.utils.data import DataLoader
from torchaudio.datasets import SPEECHCOMMANDS, LIBRISPEECH
from torchaudio.transforms import MFCC

import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
audio_backend = "soundfile"
device = "cuda" if torch.cuda.is_available() else "cpu"
num_workers = 0
pin_memory = False
non_blocking = pin_memory

excluded_dir = ["_background_noise_"]
folder_speechcommands = './SpeechCommands/speech_commands_v0.02'

char_null = "-"
char_pad = " "
# labels = [char_null, char_pad] + [d for d in next(os.walk(folder_speechcommands))[1] if d not in excluded_dir]
labels = [char_null + char_pad + string.ascii_lowercase]

# vocab_size = len(labels) + 2
shuffle = True
drop_last = True

# audio, self.sr, window_stride=(160, 80), fft_size=512, num_filt=20, num_coeffs=13
n_mfcc = 13
melkwargs = {
    'n_fft': 512,
    'n_mels': 20,
    'hop_length': 80,
}
sample_rate = 16000

batch_size = 1024  # max number of sentences per batch
optimizer_params = {
    "lr": 1.0,
    "eps": 1e-8,
    "rho": 0.95,
    "weight_decay": .1,
}

# hidden_size = 128
# num_layers = 3
hidden_size = 8
num_layers = 1

max_epoch = 200
mod_epoch = 10
clip_norm = 0.  # 10.

training_percentage = 80.
validation_percentage = 10.
MAX_NUM_WAVS_PER_CLASS = 2**27 - 1  # ~134M

dtstamp = datetime.now().strftime("%y%m%d.%H%M%S")

In [ ]:
print(dtstamp)

In [ ]:
torch.cuda.empty_cache()

# Profiling performance
pr = cProfile.Profile()
pr.enable()

torchaudio.set_audio_backend(audio_backend)
mfcc = MFCC(sample_rate=sample_rate, n_mfcc=n_mfcc, melkwargs=melkwargs).to(device)

# Text encoding

In [ ]:
class Coder:
    def __init__(self, labels):
        self.max_length = max(map(len, labels))

        labels = list(collections.OrderedDict.fromkeys(list("".join(labels))))
        self.length = len(labels)
        enumerated = list(enumerate(labels))
        flipped = [(sub[1], sub[0]) for sub in enumerated]

        d1 = collections.OrderedDict(enumerated)
        d2 = collections.OrderedDict(flipped)
        self.mapping = {**d1, **d2}

    def _map_and_pad(self, iterable, fillwith):
        iterable = [self.mapping[i] for i in iterable]  # map with dict
        iterable += [fillwith] * (self.max_length-len(iterable))  # add padding
        return iterable

    def encode(self, iterable):
        if isinstance(iterable[0], list):
            return [self.encode(i) for i in iterable]
        else:
            return self._map_and_pad(iterable, self.mapping[char_pad])

    def decode(self, tensor):
        if isinstance(tensor[0], list):
            return [self.decode(t) for t in tensor]
        else:
            return "".join(self._map_and_pad(tensor, self.mapping[1]))


coder = Coder(labels)
encode = coder.encode
decode = coder.decode
vocab_size = coder.length

# Dataset

In [ ]:
class IterableMemoryCache:
    def __init__(self, iterable):
        self.iterable = iterable
        self.iter = iter(iterable)
        self.done = False
        self.vals = []

    def __iter__(self):
        if self.done:
            return iter(self.vals)
        # chain vals so far & then gen the rest
        return itertools.chain(self.vals, self._gen_iter())

    def _gen_iter(self):
        # gen new vals, appending as it goes
        for new_val in self.iter:
            self.vals.append(new_val)
            yield new_val
        self.done = True

    def __len__(self):
        return len(self.iterable)


class MapMemoryCache(torch.utils.data.Dataset):
    """
    Wrap a dataset so that, whenever a new item is returned, it is saved to memory.
    """

    def __init__(self, dataset):
        self.dataset = dataset
        self._id = id(self)
        self._cache = [None] * len(dataset)

    def __getitem__(self, n):
        if self._cache[n]:
            return self._cache[n]

        item = self.dataset[n]
        self._cache[n] = item

        return item

    def __len__(self):
        return len(self.dataset)

In [ ]:
# @torch.jit.script
def process_datapoint(item):
    transformed = item[0].to(device, non_blocking=non_blocking)
    target = item[2].lower()
    # target = "".join(filter(str.isalnum, target))
    target = "".join(c for c in target if c.isalnum() or c == char_pad)
    # pick first channel, apply mfcc, tranpose for pad_sequence
    transformed = mfcc(transformed)
    transformed = transformed[0, ...].transpose(0, -1)
    # transformed = transformed.view(-1, 1)
    target = encode(target)
    target = torch.tensor(target, dtype=torch.long, device=transformed.device)
    return transformed, target


class PROCESSED_LIBRISPEECH(LIBRISPEECH):

    def __getitem__(self, n):
        try:
            item = super().__getitem__(n)
            return process_datapoint(item)
        except (FileNotFoundError, RuntimeError):
            return None

    def __next__(self):
        try:
            item = super().__next__()
            return process_datapoint(item)
        except (FileNotFoundError, RuntimeError):
            return self.__next__()


def datasets():
    root = "./"

    training = PROCESSED_LIBRISPEECH(root, url="train-clean-100", download=True)
    training = MapMemoryCache(training)
    validation = PROCESSED_LIBRISPEECH(root, url="dev-clean", download=True)
    validation = MapMemoryCache(validation)

    return training, validation, None

In [ ]:
def collate_fn(batch):

    tensors = [b[0] for b in batch if b]
    targets = [b[1] for b in batch if b]

    # input_lengths = [t.shape[0] for t in tensors]
    # target_lengths = [len(t) for t in targets]

    targets = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True)
    tensors = torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)
    tensors = tensors.transpose(1, -1)

    return tensors, targets  # , input_lengths, target_lengths

# Model

[Wav2Letter](https://github.com/LearnedVector/Wav2Letter/blob/master/Google%20Speech%20Command%20Example.ipynb)

In [ ]:
class PrintLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        print(x)
        return x
    

class Wav2Letter(nn.Module):
    """Wav2Letter Speech Recognition model
        https://arxiv.org/pdf/1609.03193.pdf
        This specific architecture accepts mfcc or power spectrums speech signals

        Args:
            num_features (int): number of mfcc features
            num_classes (int): number of unique grapheme class labels
    """

    def __init__(self, num_features, num_classes):
        super().__init__()

        # Conv1d(in_channels, out_channels, kernel_size, stride)
        self.layers = nn.Sequential(
            # PrintLayer(),
            nn.Conv1d(num_features, 250, 48, 2),
            nn.ReLU(),
            nn.Conv1d(250, 250, 7),
            nn.ReLU(),
            nn.Conv1d(250, 250, 7),
            nn.ReLU(),
            nn.Conv1d(250, 250, 7),
            nn.ReLU(),
            nn.Conv1d(250, 250, 7),
            nn.ReLU(),
            nn.Conv1d(250, 250, 7),
            nn.ReLU(),
            # nn.Conv1d(250, 250, 7),
            # nn.ReLU(),
            # nn.Conv1d(250, 250, 7),
            # nn.ReLU(),
            nn.Conv1d(250, 2000, 32),
            nn.ReLU(),
            nn.Conv1d(2000, 2000, 1),
            nn.ReLU(),
            nn.Conv1d(2000, num_classes, 1),
        )

    def forward(self, batch):
        """Forward pass through Wav2Letter network than
            takes log probability of output
        Args:
            batch (int): mini batch of data
            shape (batch, num_features, frame_len)
        Returns:
            Tensor with shape (batch_size, num_classes, output_len)
        """
        # print(batch.shape)
        # y_pred shape (batch_size, num_classes, output_len)
        y_pred = self.layers(batch)

        # compute log softmax probability on graphemes
        log_probs = nn.functional.log_softmax(y_pred, dim=1)
        log_probs = log_probs.transpose(1, 2).transpose(0, 1)

        # print(log_probs.shape)
        return log_probs


class BiLSTM(nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.directions = 2
        # self.layers = nn.GRU(num_features, hidden_size, num_layers=3, batch_first=True, bidirectional=True)
        # self.layers = nn.LSTM(num_features, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        # https://discuss.pytorch.org/t/lstm-to-bi-lstm/12967
        # self.lstm = nn.LSTM(num_features, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.lstm = nn.LSTM(num_features, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.hidden2class = nn.Linear(self.directions*hidden_size, num_classes)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers * num_directions, minibatch_size, hidden_dim)
        return (torch.autograd.Variable(torch.zeros(self.directions*num_layers, batch_size, hidden_size)).to(device),
                torch.autograd.Variable(torch.zeros(self.directions*num_layers, batch_size, hidden_size)).to(device))

    def forward(self, inputs):
        inputs = inputs.transpose(-1, -2)
        # outputs, _ = self.layers(inputs)
        # print(inputs.shape)
        outputs, self.hidden = self.lstm(inputs, self.hidden)
        self.hidden = (self.hidden[0].detach(), self.hidden[1].detach())
        # outputs = outputs.view(batch_size, 2*hidden_size, -1)
        outputs = self.hidden2class(outputs)

        # print(outputs.shape)
        log_probs = nn.functional.log_softmax(outputs, dim=1)
        log_probs = log_probs.transpose(0, 1)
        # print(log_probs.shape)
        return log_probs

# Word Decoder

In [ ]:
def greedy_decoder(outputs):
    """Greedy Decoder. Returns highest probability of class labels for each timestep

    Args:
        outputs (torch.Tensor): shape (input length, batch size, number of classes (including blank))

    Returns:
        torch.Tensor: class labels per time step.
    """
    _, indices = topk(outputs, k=1, dim=-1)
    return indices[..., 0]

# Train

In [ ]:
model = Wav2Letter(n_mfcc, vocab_size)
# model = BiLSTM(1, vocab_size)
# model = BiLSTM(n_mfcc, vocab_size)

In [ ]:
training, validation, _ = datasets()
loader_training = DataLoader(
    training, batch_size=batch_size, collate_fn=collate_fn, shuffle=shuffle, drop_last=drop_last,
    num_workers=num_workers, pin_memory=pin_memory,
)
loader_validation = DataLoader(
    validation, batch_size=batch_size, collate_fn=collate_fn, shuffle=False, drop_last=drop_last,
    num_workers=num_workers, pin_memory=pin_memory,
)

# model = torch.jit.script(model)
model = model.to(device, non_blocking=non_blocking)

optimizer = Adadelta(model.parameters(), **optimizer_params)
criterion = torch.nn.CTCLoss()

best_loss = 1.

In [ ]:
def forward(inputs, targets):

    inputs = inputs.to(device, non_blocking=non_blocking)
    targets = targets.to(device, non_blocking=non_blocking)
    # print(inputs.shape)
    # print(targets.shape)
    outputs = model(inputs)
    # print(outputs.shape)

    this_batch_size = len(inputs)
    input_lengths = torch.full(
        (this_batch_size,), outputs.shape[0], dtype=torch.long, device=outputs.device
    )
    target_lengths = torch.tensor(
        [target.shape[0] for target in targets], dtype=torch.long, device=targets.device
    )

    # CTC
    # https://pytorch.org/docs/master/nn.html#torch.nn.CTCLoss
    # https://discuss.pytorch.org/t/ctcloss-with-warp-ctc-help/8788/3
    # outputs: input length, batch size, number of classes (including blank)
    # targets: batch size, max target length
    # input_lengths: batch size
    # target_lengths: batch size
    return criterion(outputs, targets, input_lengths, target_lengths)

In [ ]:
sum_loss_training = []
sum_loss_validation = []

for epoch in range(max_epoch):

    model.train()

    sum_loss = 0.
    for inputs, targets in tqdm(loader_training):

        loss = forward(inputs, targets)
        sum_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        if clip_norm > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
        optimizer.step()

    # Average loss
    sum_loss = sum_loss / len(loader_training)
    sum_loss_training.append((epoch, sum_loss))
    sum_loss_str = f"{epoch}: {sum_loss:.5f}"
        
    with torch.no_grad():

        if not epoch % mod_epoch:

            # Switch to evaluation mode
            model.eval()
        
            output = model(inputs)[:, 0, :]
            output = greedy_decoder(output)
            target = decode(targets.tolist()[0])
            output = decode(output.tolist())
            print(f"{epoch}: {target}, {output} for train")
        
            sum_loss = 0.
            for inputs, targets in loader_validation:

                loss = forward(inputs, targets)
                sum_loss += loss.item()

            # Average loss
            sum_loss = sum_loss / len(loader_validation)
            sum_loss_validation.append((epoch, sum_loss))
            sum_loss_str += f", {sum_loss:.5f}"

            # Example
            output = model(inputs)[:, 0, :]
            output = greedy_decoder(output)
            target = decode(targets.tolist()[0])
            output = decode(output.tolist())
            print(f"{epoch}: {target}, {output} for validation")

            print(sum_loss_str)

            if (sum_loss < best_loss).all():
                # Save model
                torch.save(model.state_dict(), f"./model.{dtstamp}.{epoch}.ph")
                best_loss = sum_loss

In [ ]:
plt.plot(sum_loss_training, label="training")
plt.plot(sum_loss_validation, label="validation")

plt.legend()
plt.show()

In [ ]:
# Save model
torch.save(model.state_dict(), f"./model.{dtstamp}.{epoch}.ph")

# Print performance
pr.disable()
s = StringIO()
ps = pstats.Stats(pr, stream=s).strip_dirs().sort_stats("cumtime").print_stats(20)
print(s.getvalue())